# The file for training the model

In [1]:
import argparse
import os

In [2]:
def get_args_teeth():
    arguments = argparse.Namespace(epochs=10,
                                   log_dir='logs/my_run',
                                   image_size=224,
                                   dir_for_saving_images="visualization_results", #visualization results of the part prototypes
                                   seed=42,
                                   batch_size=64,
                                   gpu_ids="",
                                   disable_cuda =None,
                                   dataset="teeth",
                                   validation_size=0.,
                                   num_workers=8,
                                   weighted_loss=False,
                                   batch_size_pretrain=128,
                                   net="convnext_tiny_26",
                                   disable_pretrained=False,
                                   num_features=0, #can be increased, this determines the number of prototypes, if set to 0 it will be same as backbone
                                   bias=False,
                                   lr=0.05,
                                   lr_block=0.00005,
                                   lr_net=0.0005,
                                   optimizer="Adam",
                                   weight_decay=0.0,
                                   epochs_pretrain=10,
                                   state_dict_dir_net="",
                                   freeze_epoch=10,

                                    )
    return arguments

arguments =get_args_teeth()


In [ ]:
import main
main.run_pipnet(arguments)

Log dir:  logs/my_run
Device used:  cpu with id []
Num classes (k) =  3 ['filling', 'impacted', 'implant'] etc.
Classes:  {'filling': 0, 'impacted': 1, 'implant': 2}
Number of prototypes:  768
chosen network is convnext
Classification layer initialized with mean 1.0001195669174194
Output shape:  torch.Size([64, 768, 26, 26])

Pretrain Epoch 1 with batch size 128


Pretrain Epoch1:   0% 0/13 [00:00<?, ?it/s]

Number of parameters that require gradient:  31
Align weight:  0.1 , U_tanh weight:  5.0 Class weight: 0.0
Pretrain? True Finetune? False


Pretrain Epoch1:  54% 7/13 [15:24<13:30, 135.11s/it, L: 5.281, LA:6.43, LT:0.927, num_scores>0.1:2.8]

In [3]:
def get_args_teeth_command() -> argparse.Namespace:
    """
    this only works through the command line
    """
    parser = argparse.ArgumentParser('Train a PIP-Net')
    parser.add_argument('--epochs',
                        type=int,
                        default=5,
                        help='The number of epochs PIP-Net should be trained (second training stage)')

    parser.add_argument('--log_dir',
                        type=str,
                        default='./runs/run_pipnet',
                        help='The directory in which train progress should be logged')

    parser.add_argument('--image_size',
                        type=int,
                        default=512,
                        help='Input images will be resized to --image_size x --image_size (square). Code only tested with 224x224, so no guarantees that it works for different sizes.')
    arguments = parser.parse_args()
    if len(arguments.log_dir.split('/'))>2:
        if not os.path.exists(arguments.log_dir):
            os.makedirs(arguments.log_dir)
    return arguments

usage: Train a PIP-Net [-h] [--epochs EPOCHS] [--log_dir LOG_DIR]
                       [--image_size IMAGE_SIZE]
Train a PIP-Net: error: unrecognized arguments: -f /Users/mehdigreefhorst/Library/Jupyter/runtime/kernel-11bd5591-8724-475d-8035-8128a921fb68.json


SystemExit: 2

/Users/mehdigreefhorst/Desktop/PIPNet/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
def get_args() -> argparse.Namespace:

    parser = argparse.ArgumentParser('Train a PIP-Net')
    parser.add_argument('--dataset',
                        type=str,
                        default='teeth',
                        help='Data set on PIP-Net should be trained')
    parser.add_argument('--validation_size',
                        type=float,
                        default=0.,
                        help='Split between training and validation set. Can be zero when there is a separate test or validation directory. Should be between 0 and 1. Used for partimagenet (e.g. 0.2)')
    parser.add_argument('--net',
                        type=str,
                        default='convnext_tiny_26',
                        help='Base network used as backbone of PIP-Net. Default is convnext_tiny_26 with adapted strides to output 26x26 latent representations. Other option is convnext_tiny_13 that outputs 13x13 (smaller and faster to train, less fine-grained). Pretrained network on iNaturalist is only available for resnet50_inat. Options are: resnet18, resnet34, resnet50, resnet50_inat, resnet101, resnet152, convnext_tiny_26 and convnext_tiny_13.')
    parser.add_argument('--batch_size',
                        type=int,
                        default=64,
                        help='Batch size when training the model using minibatch gradient descent. Batch size is multiplied with number of available GPUs')
    parser.add_argument('--batch_size_pretrain',
                        type=int,
                        default=128,
                        help='Batch size when pretraining the prototypes (first training stage)')
    parser.add_argument('--epochs',
                        type=int,
                        default=5,
                        help='The number of epochs PIP-Net should be trained (second training stage)')
    parser.add_argument('--epochs_pretrain',
                        type=int,
                        default = 10,
                        help='Number of epochs to pre-train the prototypes (first training stage). Recommended to train at least until the align loss < 1'
                        )
    parser.add_argument('--optimizer',
                        type=str,
                        default='Adam',
                        help='The optimizer that should be used when training PIP-Net')
    parser.add_argument('--lr',
                        type=float,
                        default=0.05,
                        help='The optimizer learning rate for training the weights from prototypes to classes')
    parser.add_argument('--lr_block',
                        type=float,
                        default=0.0005,
                        help='The optimizer learning rate for training the last conv layers of the backbone')
    parser.add_argument('--lr_net',
                        type=float,
                        default=0.0005,
                        help='The optimizer learning rate for the backbone. Usually similar as lr_block.')
    parser.add_argument('--weight_decay',
                        type=float,
                        default=0.0,
                        help='Weight decay used in the optimizer')
    parser.add_argument('--disable_cuda',
                        action='store_true',
                        help='Flag that disables GPU usage if set')
    parser.add_argument('--log_dir',
                        type=str,
                        default='./runs/run_pipnet',
                        help='The directory in which train progress should be logged')
    parser.add_argument('--num_features',
                        type=int,
                        default = 0,
                        help='Number of prototypes. When zero (default) the number of prototypes is the number of output channels of backbone. If this value is set, then a 1x1 conv layer will be added. Recommended to keep 0, but can be increased when number of classes > num output channels in backbone.')
    parser.add_argument('--image_size',
                        type=int,
                        default=224,
                        help='Input images will be resized to --image_size x --image_size (square). Code only tested with 224x224, so no guarantees that it works for different sizes.')
    parser.add_argument('--state_dict_dir_net',
                        type=str,
                        default='',
                        help='The directory containing a state dict with a pretrained PIP-Net. E.g., ./runs/run_pipnet/checkpoints/net_pretrained')
    parser.add_argument('--freeze_epochs',
                        type=int,
                        default = 10,
                        help='Number of epochs where pretrained features_net will be frozen while training classification layer (and last layer(s) of backbone)'
                        )
    parser.add_argument('--dir_for_saving_images',
                        type=str,
                        default='visualization_results',
                        help='Directoy for saving the prototypes and explanations')
    parser.add_argument('--disable_pretrained',
                        action='store_true',
                        help='When set, the backbone network is initialized with random weights instead of being pretrained on another dataset).'
                        )
    parser.add_argument('--weighted_loss',
                        action='store_true',
                        help='Flag that weights the loss based on the class balance of the dataset. Recommended to use when data is imbalanced. ')
    parser.add_argument('--seed',
                        type=int,
                        default=1,
                        help='Random seed. Note that there will still be differences between runs due to nondeterminism. See https://pytorch.org/docs/stable/notes/randomness.html')
    parser.add_argument('--gpu_ids',
                        type=str,
                        default='',
                        help='ID of gpu. Can be separated with comma')
    parser.add_argument('--num_workers',
                        type=int,
                        default=8,
                        help='Num workers in dataloaders.')
    parser.add_argument('--bias',
                        action='store_true',
                        help='Flag that indicates whether to include a trainable bias in the linear classification layer.'
                        )
    parser.add_argument('--extra_test_image_folder',
                        type=str,
                        default='./experiments',
                        help='Folder with images that PIP-Net will predict and explain, that are not in the training or test set. E.g. images with 2 objects or OOD image. Images should be in subfolder. E.g. images in ./experiments/images/, and argument --./experiments')

    args = parser.parse_args()
    if len(args.log_dir.split('/'))>2:
        if not os.path.exists(args.log_dir):
            os.makedirs(args.log_dir)


    return args